# Structured Output

This notebook demonstrates two methods for getting structured output from LLMs:

1. **Prompt Engineering**: Including output format instructions directly in the prompt
2. **Function Calling**: Using the model's built-in structured output capabilities

We'll use joke generation as our example task to compare these approaches.

In [ ]:
from dotenv import load_dotenv
from rich import print

load_dotenv(verbose=True)

### Method 1: Prompt Engineering

This approach uses LangChain's `PydanticOutputParser` to:

1. Define our desired output structure using Pydantic
2. Automatically generate format instructions
3. Inject those instructions into the prompt

The model then returns text that we parse into our structured format.

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

The usual "tell me a joke" LLM call...

In [ ]:
from genai_tk.core.llm_factory import get_llm
from genai_tk.core.prompts import def_prompt


class Joke(BaseModel):
    the_joke: str = Field(description="a good joke")
    explanation: str = Field(description="explain why it's funny in exactly 10 words")
    rate: float = Field(description="rate how the joke is funny between 0 and 5")


parser = PydanticOutputParser(pydantic_object=Joke)

prompt_with_format = """
    tell me  a joke on {topic}
    ---
    {format_instructions}"""

structured_prompt = def_prompt(user=prompt_with_format).partial(
    format_instructions=parser.get_format_instructions(),
)

LLM_ID = None
structured_joke = structured_prompt | get_llm(llm_id=LLM_ID) | parser

r = structured_joke.invoke({"topic": "cat"})
print(r)

In [ ]:
from devtools import debug

debug(r)

In [ ]:
print(parser.get_format_instructions())

In [ ]:
print(structured_prompt)

In [ ]:
# You can have a look at the generated prompt:
print(structured_prompt.invoke({"topic": "cat"}).messages[0].content)

### Method 2: Function Calling

This approach uses the model's native structured output capabilities via:

1. The `with_structured_output()` method
2. The same Pydantic schema for type safety
3. Fewer prompt instructions needed

This typically works better with models that support function calling (like GPT-4).

In [ ]:
prompt = "tell me  a joke on {topic}"

# MODEL = None
MODEL = "gpt_4omini_edenai"  # better with Azure
chain = def_prompt(prompt) | get_llm(llm_id=MODEL).with_structured_output(Joke)
print(chain.invoke({"topic": "cat"}))

## Assignment: Joke Rating System

Let's extend our structured output by:

1. Creating an Enum to represent joke ratings
2. Modifying our Joke class to include this rating
3. Using LangChain's Enum output parser

Reference: [LangChain Enum Output Parser](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/enum/)

In [ ]:
from enum import Enum


class JokeRater(Enum):
    NOT_SO_GOOD = 0
    GOOD = 1
    VERY_GOOD = 2